In [1]:
import numpy as np  
import torch
# import sys
# import subprocess
# sys.path.append("/home/talax/xtof/local/Mod/lib64")
# from mod import *
# import mod
# from utils import smarts_to_gml_with_mapping
import random
# from rdkit.Chem import AllChem,Draw

from rdkit import Chem
# from rdchiral.main import rdchiralRun, rdchiralReaction, rdchiralReactants
import pandas as pd 
# import torch
# from utils import smarts_to_gml

/tmp/ipykernel_1346515/1711071717.py:14: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
device

'cpu'

In [42]:
from utils import collect_rules
rules_path = "/home/mescalin/yitao/Documents/Code/3HPspace"
rules = collect_rules(rules_path)
rule = rules[0]

for v in rule.vertices:
    # print(v.id)
    print(v.left.stringLabel)

    # print(v.right)
    # print(v.id)

for edge in rule.edges:
    print(edge.source.id)
    print(edge.target.id)
    print(edge.left.stringLabel)
    # print(edge.right.stringLabel)

C
C
O
C
O
O
H
H
NAD+
0
1
-
1
2
-
1
3
-
1
7
-
2
6
-
3
4
=
3
5
-
7
8


LogicError: Can not get string label on a null edge.

In [2]:
""" 

    Xe : indicate CoA group
    U : indicate NAD group
    Rn : indicate R group
"""

RULES  = [["[CH3:1][C:2]([H:8])([O:3][H:7])[C:4](=[O:5])[OH:6].[*+:9]>>[CH3:1][C:2](=[O:3])[C:4](=[O:5])[OH:6].[H+:7].[H:8][*:9]","(S)-lactate + NAD+ = pyruvate + NADH + H+","1.1.1.27"],
      ["[CH3:1][C:2]([H:6])=[O:3].[*+:5].[S:4]([H:8])[*:7]>>[CH3:1][C:2](=[O:3])[S:4][*:7].[H+:8].[*:5][H:6]","acetaldehyde + CoA + NAD+ = acetyl-CoA + NADH + H+","1.2.1.10"],
      ["[H:20][N:1]([H:21])[C:2]([CH2:3][C:4](=[O:5])[OH:6])([H:22])([C:7](=[O:8])[OH:9]).[O:10]=[C:11]([OH:12])[CH2:13][CH2:14][C:15](=[O:16])[C:17](=[O:18])[OH:19]>>[C:2]([CH2:3][C:4](=[O:5])[OH:6])([C:7](=[O:8])[OH:9])=[O:16].[H:20][N:1]([H:21])[C:15]([CH2:14][CH2:13][C:11](=[O:10])[OH:12])([H:22])[C:17](=[O:18])[OH:19]","L-Aspartate + 2-Oxoglutarate <=> Oxaloacetate + L-Glutamate","2.6.1.1"],
      ["[CH3:1](=[O:2])[S:3][*:5].[H:7][O:4][H:6]>>[CH3:1](=[O:2])[O:4][H:7].[H:6][S:3][*:5]","Acyl-CoA + H2O <=> CoA + Carboxylate","3.1.2.20"],
      ["[CH3:1][S+:2]([CH2:3][CH2:4][CH:5]([H:11])[NH2:6])[*:7].[O:8]=[C:9]=[O:10]>>[CH3:1][S+:2]([CH2:3][CH2:4][C@H:5]([NH2:6])[C:9](=[O:10])[O:8][H:11])[*:7]","S-adenosyl-L-methionine = (5-deoxy-5-adenosyl)(3-aminopropyl)methylsulfonium salt + CO2","4.1.1.50"],
      ["[O:1]=[C:2]([OH:3])[CH:4]([H:11])[C@H:5]([O:6][H:10])[C:7](=[O:8])[OH:9]>>[O:1]=[C:2]([OH:3])/[CH:4]=[CH:5]/[C:7](=[O:8])[OH:9].[H:10][O:6][H:11]","fumarate + H2O = L-malate","4.2.1.2"],
      ["[H:8][CH2:1][CH:2]([O:3][H:9])[C:4](=[O:5])[S:6][*:7]>>[CH2:1]=[CH:2][C:4](=[O:5])[S:6][*:7].[H:8][O:3][H:9]","Lactoyl-CoA <=> Propenoyl-CoA + H2O","4.2.1.54"],
      ["[H:13][NH2:1][C@@H:2]([CH:3]([H:10])[C:4](=[O:5])[OH:6])[C:7](=[O:8])[OH:9]>>[CH:2](=[CH:3]/[C:4](=[O:5])[OH:6])\\[C:7](=[O:8])[OH:9].[NH2:1][H:10]","L-Aspartate <=> Fumarate + Ammonia","4.3.1.1"],
      ["[NH2:1][CH2:2][CH2:3][CH2:4][CH:5]([H:11])[C@H:6]([NH2:7])[C:8](=[O:9])[OH:10]>>[NH2:1][CH2:2][CH2:3][CH2:4][C@H:5]([NH2:7])[CH:6]([H:11])[C:8](=[O:9])[OH:10]","L-lysine = (3S)-3,6-diaminohexanoate","5.4.3.2"],
      ["[CH3:1][C:2](=[O:3])[OH:4].[Xe:18][S:5][H:19].[CH3:6][O:7][P:8](=[O:9])([OH:10])[O:11][P:12](=[O:13])([OH:14])[O:15][P:16](=[O:17])([OH:20])[OH:21]>>[CH3:1][C:2](=[O:3])[S:5][Xe:18].[OH:4][P:8]([O:7][CH3:6])(=[O:9])[OH:10].[O:11]([H:19])[P:12](=[O:13])([OH:14])[O:15][P:16](=[O:17])([OH:20])[OH:21]","ATP + Acetate + CoA <=> AMP + Diphosphate + Acetyl-CoA","6.2.1.1"],
      ["[H:23][CH2:1][C:2](=[O:3])[S:4][Xe:24].[CH3:5][O:6][P:7](=[O:8])([OH:9])[O:10][P:11](=[O:12])([OH:13])[O:14][P:15](=[O:16])([OH:17])[OH:18].[O:19]=[C:20]([OH:21])[OH:22]>>[CH2:1]([C:2](=[O:3])[S:4][Xe:24])[C:20](=[O:19])[OH:22].[CH3:5][O:6][P:7](=[O:8])([OH:9])[O:10][P:11](=[O:12])([OH:13])[OH:21].[H:23][O:14][P:15](=[O:16])([OH:17])[OH:18]","ATP + acetyl-CoA + HCO3- = ADP + malonyl-CoA + phosphate ","6.4.1.2"]
      ["[NH2:1][C:2](=[O:3])[c:4]1[cH:5][cH:6][cH:7][n+:8]([C@@H:9]2[O:10][C@H:11]([CH2:12][O:13][P:14](=[O:15])([OH:16])[O:17][P:18](=[O:19])([OH:20])[O:21][CH2:22][C@H:23]3[O:24][C@@H:25]([n:26]4[cH:27][n:28][c:29]5[c:30]([NH2:31])[n:32][cH:33][n:34][c:35]45)[C@H:36]([OH:37])[C@@H:38]3[OH:39])[C@@H:40]([OH:41])[C@H:42]2[OH:43])[cH:44]1.[NH2:45][C@@H:46]([CH2:47][CH2:48][C:49](=[O:50])[OH:51])[C:52](=[O:53])[OH:54].[OH2:55]>>[C:46]([CH2:47][CH2:48][C:49](=[O:50])[OH:51])([C:52](=[O:53])[OH:54])=[O:55].[H+].[NH2:1][C:2](=[O:3])[C:4]1=[CH:44][N:8]([C@@H:9]2[O:10][C@H:11]([CH2:12][O:13][P:14](=[O:15])([OH:16])[O:17][P:18](=[O:19])([OH:20])[O:21][CH2:22][C@H:23]3[O:24][C@@H:25]([n:26]4[cH:27][n:28][c:29]5[c:30]([NH2:31])[n:32][cH:33][n:34][c:35]45)[C@H:36]([OH:37])[C@@H:38]3[OH:39])[C@@H:40]([OH:41])[C@H:42]2[OH:43])[CH:7]=[CH:6][CH2:5]1.[NH3:45]"]]

# RULES =[["[CH3:1][C:2](=[O:3])[S:4][Xe:1000].[NH2:5][c:6]1[cH:7][cH:8][c:9]([C:10](=[O:11])[OH:12])[cH:13][cH:14]1>>[S:4]([H:1001])[Xe:1000].[CH3:1][C:2](=[O:3])[NH:5][c:6]1[cH:7][cH:8][c:9]([C:10](=[O:11])[OH:12])[cH:13][cH:14]1","acetyl-CoA + 4-aminobenzoic acid = CoA + N-acetyl-4-aminobenzoate","2.3.1.50"],
#       ["[CH3:1][C:2](=[O:3])[S:4][Xe:1000].[NH2:5][c:6]1[cH:7][cH:8][c:9]([OH:10])[cH:11][cH:12]1>>[S:4]([H:1001])[Xe:1000].[CH3:1][C:2](=[O:3])[NH2:5][c:6]1[cH:7][cH:8][c:9]([OH:10])[cH:11][cH:12]1","acetyl-CoA + 4-amimophenol = CoA + paracetamol","2.3.1.118"],
# ["[H+].[H:1000][U:1001].[NH2:1][c:2]1[cH:3][cH:4][c:5]([C:6](=[O:7])[OH:8])[cH:9][cH:10]1.[O:11]=[O:12]>>[C:6](=[O:7])=[O:8].[U+:1001].[OH2:12].[NH2:1][c:2]1[cH:3][cH:4][c:5]([OH:11])[cH:9][cH:10]1","4-aminobenzoate + NAD(P)H + H+ + O2 = 4-aminophenol + NAD(P)+ + H2O + CO2","1.14.13.27"],]



RULES_ARR = np.array(RULES)
rules_set = RULES_ARR[:,:2]
RULES_DF = pd.DataFrame(RULES,columns=["mapped","orig_rxn_text","EC"])
RULES_DF 

,mapped,orig_rxn_text,EC
0,[CH3:1][C:2]([H:8])([O:3][H:7])[C:4](=[O:5])[O...,(S)-lactate + NAD+ = pyruvate + NADH + H+,1.1.1.27
1,[CH3:1][C:2]([H:6])=[O:3].[*+:5].[S:4]([H:8])[...,acetaldehyde + CoA + NAD+ = acetyl-CoA + NADH ...,1.2.1.10
2,[H:20][N:1]([H:21])[C:2]([CH2:3][C:4](=[O:5])[...,L-Aspartate + 2-Oxoglutarate <=> Oxaloacetate ...,2.6.1.1
3,[CH3:1](=[O:2])[S:3][*:5].[H:7][O:4][H:6]>>[CH...,Acyl-CoA + H2O <=> CoA + Carboxylate,3.1.2.20
4,[CH3:1][S+:2]([CH2:3][CH2:4][CH:5]([H:11])[NH2...,S-adenosyl-L-methionine = (5-deoxy-5-adenosyl)...,4.1.1.50
5,[O:1]=[C:2]([OH:3])[CH:4]([H:11])[C@H:5]([O:6]...,fumarate + H2O = L-malate,4.2.1.2
6,[H:8][CH2:1][CH:2]([O:3][H:9])[C:4](=[O:5])[S:...,Lactoyl-CoA <=> Propenoyl-CoA + H2O,4.2.1.54
7,[H:13][NH2:1][C@@H:2]([CH:3]([H:10])[C:4](=[O:...,L-Aspartate <=> Fumarate + Ammonia,4.3.1.1
8,[NH2:1][CH2:2][CH2:3][CH2:4][CH:5]([H:11])[C@H...,"L-lysine = (3S)-3,6-diaminohexanoate",5.4.3.2
9,[CH3:1][C:2](=[O:3])[OH:4].[Xe:18][S:5][H:19]....,ATP + Acetate + CoA <=> AMP + Diphosphate + Ac...,6.2.1.1


In [7]:
import pandas as pd
import random

def extract_rule_from_brenda(brenda_csv_file: str, ec_num_list: list) -> pd.DataFrame:
    # Read the BRENDA dataset CSV file into a DataFrame
    df = pd.read_csv(brenda_csv_file)  # Adjust separator as per the actual file format

    # Print the column names for debugging
    # print("Columns in the DataFrame:", df.columns)
  
    # Strip any whitespace from column names
    df.columns = df.columns.str.strip()
    # print(df.columns)
    # print(df)
    # Ensure 'ec_num' column exists
    if 'ec_num' not in df.columns:
        raise KeyError("The 'ec_num' column is not found in the CSV file. Please check the column names.")

    # Filter the dataframe to include only the relevant EC numbers
    filtered_df = df[df['ec_num'].isin(ec_num_list)]
    
    # Create a result dataframe to store the output
    result_data = []

    # Process each EC number in the input list
    for ec_num in ec_num_list:
        # Filter the rows for the current EC number
        ec_df = filtered_df[filtered_df['ec_num'] == ec_num]

        # Prioritize "Escherichia coli" organism, if not available, use "Homo sapiens"
        organism_df = ec_df[ec_df['organism'] == "Escherichia coli"]
        if organism_df.empty:
            organism_df = ec_df[ec_df['organism'] == "Homo sapiens"]

        # If still empty, use all available organisms for the current EC number
        if organism_df.empty:
            organism_df = ec_df

        # Select a random row from the filtered data for the current EC number
        if not organism_df.empty:
            selected_row = organism_df.sample(n=1).iloc[0]
            result_data.append({
                'mapped': selected_row['mapped'],
                'orig_rxn_text': selected_row['orig_rxn_text'],
                'organism': selected_row['organism'],
                'ec_num': selected_row['ec_num']
            })

    # Create a DataFrame from the result data
    result_df = pd.DataFrame(result_data)

    return result_df


In [8]:
brenda_csv_file = "/home/mescalin/yitao/Documents/Datasets/processed_reactions_enzymemap_hesther.csv"
ec_list = ["1.4.1.2","4.1.1.1","4.1.1.33","5.4.99.2"]
r = extract_rule_from_brenda(brenda_csv_file,ec_list)
r

,mapped,orig_rxn_text,organism,ec_num
0,[NH2:1][C:2](=[O:3])[c:4]1[cH:5][cH:6][cH:7][n...,L-glutamate + NAD+ + H2O = 2-oxoglutarate + NA...,Escherichia coli,1.4.1.2
1,[CH3:1][C:2](=[O:3])[C:4](=[O:5])[OH:6]>>[C:4]...,pyruvate = acetaldehyde + CO2 {r},Komagataella pastoris,4.1.1.1
2,[CH3:1][C@@:2]([OH:3])([CH2:4][CH2:5][O:6][P:7...,ATP + 5-diphosphomevalonate = ADP + phosphate ...,Homo sapiens,4.1.1.33
3,[CH3:1][C@H:2]([C:3](=[O:4])[OH:5])[C:6](=[O:7...,(R)-2-methylmalonyl-CoA = succinyl-CoA {r},Escherichia coli,5.4.99.2


In [24]:
rs=r
rs.to_csv('/home/mescalin/yitao/Documents/Code/3HP/ruleset_operator.csv')
rs['mapped'][1]

'[CH3:1][C:2](=[O:3])[C:4](=[O:5])[OH:6]>>[C:4](=[O:5])=[O:6].[CH3:1][CH:2]=[O:3]'

In [27]:
r1 = '[CH3:1][C:2](=[O:3])[C:4](=[O:5])[O:6][H:7]>>[C:4](=[O:5])=[O:6].[CH3:1][C:2]([H:7])=[O:3]'

In [28]:

reaction = AllChem.ReactionFromSmarts(r1)
rxn_img = Draw.ReactionToImage(reaction, subImgSize=(1200, 1200))
rxn_img.show()
# rl = smarts_to_gml([rxn,"(S)-lactate + NAD+ = pyruvate + NADH + H+"])
# print(rl)

In [17]:
mol = Chem.MolFromSmiles("[*+:1][O:2]")
for atom in mol.GetAtoms():
    print(atom.GetSymbol(),atom.GetIdx(),atom.GetFormalCharge())

    print(atom.GetAtomicNum(),atom.GetChiralTag(),atom.GetHybridization())
    print(atom.GetIsAromatic(),atom.IsInRing(),atom.GetAtomMapNum())

* 0 1
0 CHI_UNSPECIFIED UNSPECIFIED
False False 1
O 1 0
8 CHI_UNSPECIFIED SP3
False False 2


In [29]:
from utils import smarts_to_gml_v2
rules_set = pd.read_csv("/home/mescalin/yitao/Documents/Code/3HP/ruleset_operator.csv")
rule_smt = rules_set['mapped'][1]
rule_name = rules_set['orig_rxn_text'][1] + '|| ec num:  ' + rules_set['ec_num'][1]
print(rule_smt)
print(rule_name)

rule = smarts_to_gml_v2(rule_smt,rule_name)
print(rule)
# rules = []

# for r in rules_set:
#     rule = smarts_to_gml(r)
#     rules.append(rule)
#     print(rule)

# for i, r in enumerate(rules):

with open(f"/home/mescalin/yitao/Documents/Code/3HP/gml_rules/rules_00{13}.gml","w") as f:

    f.write(rule)

[CH3:1][C:2](=[O:3])[C:4](=[O:5])[O:6][H:7]>>[C:4](=[O:5])=[O:6].[CH3:1][C:2]([H:7])=[O:3]
pyruvate = acetaldehyde + CO2   {r}|| ec num:  4.1.1.1
rule[
 ruleID "pyruvate = acetaldehyde + CO2   {r}|| ec num:  4.1.1.1"
 
 left [
   edge [ source 2 target 4 label "-"]
   edge [ source 4 target 6 label "-"]
   edge [ source 6 target 7 label "-"]
 ]
 
 context [
   node [ id 1 label "C"]
   node [ id 2 label "C"]
   node [ id 3 label "O"]
   node [ id 4 label "C"]
   node [ id 5 label "O"]
   node [ id 6 label "O"]
   node [ id 7 label "H"]
   edge [ source 1 target 2 label "-"]
   edge [ source 2 target 3 label "="]
   edge [ source 4 target 5 label "="]
 ]
 
 right [
   edge [ source 2 target 7 label "-"]
   edge [ source 4 target 6 label "="]
 ]
]


In [6]:
rgml = []

for r in rules:
    rgml.append(ruleGMLString(r))


rgml

['(S)-lactate + NAD+ = pyruvate + NADH + H+'(0),
 'acetaldehyde + CoA + NAD+ = acetyl-CoA + NADH + H+'(1),
 'L-Aspartate + 2-Oxoglutarate <=> Oxaloacetate + L-Glutamate'(2),
 'Acyl-CoA + H2O <=> CoA + Carboxylate'(3),
 'S-adenosyl-L-methionine = (5-deoxy-5-adenosyl)(3-aminopropyl)methylsulfonium salt + CO2'(4),
 'fumarate + H2O = L-malate'(5),
 'Lactoyl-CoA <=> Propenoyl-CoA + H2O'(6),
 'L-Aspartate <=> Fumarate + Ammonia'(7),
 'L-lysine = (3S)-3,6-diaminohexanoate'(8),
 'ATP + Acetate + CoA <=> AMP + Diphosphate + Acetyl-CoA'(9),
 'ATP + acetyl-CoA + HCO3- = ADP + malonyl-CoA + phosphate '(10)]

In [7]:
pyruvate = smiles("CC(=O)C(=O)[O-]", "PYRUVATE")
lactate = smiles("CC(C(=O)[O-])O", "LACTATE")
# lactoyl_coa = smiles("CC(C(=O)SCCNC(=O)CCNC(=O)[C@@H](C(C)(C)COP(=O)(O)OP(=O)(O)OC[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)(O)O)O)O", "LACTOYL-COA")
# aspartate = smiles("C([C@@H](C(=O)O)N)C(=O)O", "ASPARTATE")
nadh = graphDFS("[U][H]")
nadplus = graphDFS("[U+]")
oxygen = smiles("O=O", "O2")
water = smiles("O", "H2O")
hplus = smiles("[H+]", "hplus")

In [8]:
knownMols = [pyruvate, lactate,nadh,nadplus,oxygen,water,hplus]
# lactoyl_coa, /
# aspartate
knownMols

['PYRUVATE'(0),
 'LACTATE'(1),
 'g_{2}'(2),
 'g_{3}'(3),
 'O2'(4),
 'H2O'(5),
 'hplus'(6)]

In [ ]:
# flush summary file handle
post.flushCommands()
# generate summary/summery.pdf
subprocess.run(["/home/talax/xtof/local/Mod/bin/mod_post"])

In [ ]:

eductMols = [pyruvate, lactate, nadh,nadplus, water]

strat = (addSubset(eductMols)>> repeat[5](inputRules))
ls = LabelSettings(LabelType.Term, LabelRelation.Unification)
dg = DG(graphDatabase=knownMols, labelSettings=ls)
dg

In [51]:
dg.build().execute(strat)
dg


Repeat, limit = 5
  Round 1:
  Round 1: Result subset has 0 graphs.


DG(4)

In [54]:
dg.print()


LogicError: Can not open file 'out/002_dg_4_11100.dot'.

In [4]:
import torch
from torch_geometric.nn import HeteroLinear

# Sample input data
features = torch.tensor([[1.0, 2.0],  # Person node
                         [3.0, 4.0],  # Company node
                         [5.0, 6.0],  # Person node 
                         [7.0, 8.0]]) # Location node
type_vec = torch.tensor([0, 1, 0, 2])  # 0 for Person, 1 for Company, 2 for Location

# Create a HeteroLinear layer
hetero_linear = HeteroLinear(in_channels=2, out_channels=3, num_types=3)

# Apply the transformation
output = hetero_linear(features, type_vec)
print(output)


/scr/xindus/yitao/miniforge3/envs/aibio/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[ 0.7389,  2.0832, -0.4622],
        [ 3.3329, -0.7149, -3.9373],
        [ 1.5288,  6.1248, -3.6399],
        [ 2.5763, -6.6367,  6.8477]], grad_fn=<IndexPutBackward0>)
